# Market Predictor

## Load Data

In [1]:
import numpy as np
import pandas as pd


df = pd.read_csv('./csv/000997.csv')

## Clean Data

In [2]:
df['Amp'] = df.apply(lambda row: (row['High'] - row['Low']) / row['Open'] * 100 ,axis=1)

In [44]:
clean_df = df.iloc[::-1].loc[:,['Date', 'Chg', 'TORate','Amp']][1:]
clean_df[0:].reset_index(drop=True)

,Date,Chg,TORate,Amp
0,2000-08-08,0.40,7.47,2.840909
1,2000-08-09,3.08,8.97,5.472496
2,2000-08-10,-0.16,6.69,3.217889
3,2000-08-11,-0.71,3.80,1.107726
4,2000-08-14,-1.66,3.25,1.722222
5,2000-08-15,2.05,2.09,2.760563
6,2000-08-16,-0.77,1.67,1.405733
7,2000-08-17,-0.86,1.26,2.082755
8,2000-08-18,-0.31,1.72,1.098592
9,2000-08-21,-1.15,2.51,1.380282


In [45]:
df_list = []

for i in range(10):
    df_list.append(clean_df[i:].reset_index(drop=True))

pd.concat(df_list, axis=1, join='inner', ignore_index=True)

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,2000-08-08,0.40,7.47,2.840909,2000-08-09,3.08,8.97,5.472496,2000-08-10,-0.16,...,1.26,2.082755,2000-08-18,-0.31,1.72,1.098592,2000-08-21,-1.15,2.51,1.380282
1,2000-08-09,3.08,8.97,5.472496,2000-08-10,-0.16,6.69,3.217889,2000-08-11,-0.71,...,1.72,1.098592,2000-08-21,-1.15,2.51,1.380282,2000-08-22,-2.22,2.07,2.724177
2,2000-08-10,-0.16,6.69,3.217889,2000-08-11,-0.71,3.80,1.107726,2000-08-14,-1.66,...,2.51,1.380282,2000-08-22,-2.22,2.07,2.724177,2000-08-23,-2.04,3.58,4.903678
3,2000-08-11,-0.71,3.80,1.107726,2000-08-14,-1.66,3.25,1.722222,2000-08-15,2.05,...,2.07,2.724177,2000-08-23,-2.04,3.58,4.903678,2000-08-24,5.34,2.38,8.985765
4,2000-08-14,-1.66,3.25,1.722222,2000-08-15,2.05,2.09,2.760563,2000-08-16,-0.77,...,3.58,4.903678,2000-08-24,5.34,2.38,8.985765,2000-08-25,2.23,4.57,3.954802
5,2000-08-15,2.05,2.09,2.760563,2000-08-16,-0.77,1.67,1.405733,2000-08-17,-0.86,...,2.38,8.985765,2000-08-25,2.23,4.57,3.954802,2000-08-28,-0.11,3.70,2.477974
6,2000-08-16,-0.77,1.67,1.405733,2000-08-17,-0.86,1.26,2.082755,2000-08-18,-0.31,...,4.57,3.954802,2000-08-28,-0.11,3.70,2.477974,2000-08-29,1.10,4.08,4.005525
7,2000-08-17,-0.86,1.26,2.082755,2000-08-18,-0.31,1.72,1.098592,2000-08-21,-1.15,...,3.70,2.477974,2000-08-29,1.10,4.08,4.005525,2000-08-30,3.36,14.33,5.378378
8,2000-08-18,-0.31,1.72,1.098592,2000-08-21,-1.15,2.51,1.380282,2000-08-22,-2.22,...,4.08,4.005525,2000-08-30,3.36,14.33,5.378378,2000-08-31,-4.04,5.56,6.380697
9,2000-08-21,-1.15,2.51,1.380282,2000-08-22,-2.22,2.07,2.724177,2000-08-23,-2.04,...,14.33,5.378378,2000-08-31,-4.04,5.56,6.380697,2000-09-01,-1.32,2.91,2.464009
